Load Libraries

In [9]:
import os
import pandas as pd

In [ ]:
if not os.path.exists('data-innovation-days-2025'):
  !git clone https://github.com/jeffjaylacs/data-innovation-days-2025.git
  print("Directory 'data-innovation-days-2025' created successfully.")
else:
  print("Directory 'data-innovation-days-2025' already exists. Skipping clone.")

In [ ]:
df = pd.read_csv('data-innovation-days-2025/data/train_subset.csv')
display(df.head(10))